sklearn for text vectorization and similarity calculation.

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

Loading data in

In [2]:
df = pd.read_csv('bookData.csv')


The stop_words='english' parameter tells the TfidfVectorizer to exclude common English words (such as "the", "and", "in") that appear across a wide range of documents and are typically not useful for distinguishing between different documents' content. Removing these stop words can help improve the focus on more meaningful terms within the 'Categories' data for calculating similarities between books.

In [3]:
tfidf_vectorizer = TfidfVectorizer(stop_words='english')  
tfidf_matrix = tfidf_vectorizer.fit_transform(df['Categories'])

This score will range from 0 to 1, where 1 means two books have exactly the same categories mentioned.

In [4]:
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

Code for reccomendation but without outputting categories

In [ ]:
'''
def get_recommendations(title, cosine_sim=cosine_sim):
    # Ensure the title exists in the dataset to avoid errors
    if title not in df['Title'].values:
        return "The book title you entered is not in the dataset."
    
    # Get the index of the book that matches the title
    idx = df.index[df['Title'] == title].tolist()[0]

    # Get the pairwise similarity scores of all books with that book
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the books based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Skip the first one because it's the book itself and get the scores of the top 3 similar books
    top_sim_scores = sim_scores[1:4]

    # Get the book indices and the similarity scores
    book_indices = [i[0] for i in top_sim_scores]
    similarities = [i[1] for i in top_sim_scores]

    # Format the output
    recommendations = df['Title'].iloc[book_indices]
    output = "Based on this book, the top 3 most closely related ones are:\n"
    for i, (index, similarity) in enumerate(zip(book_indices, similarities)):
        percentage_similarity = similarity * 100
        output += f"{i+1}: {df['Title'].iloc[index]} ({percentage_similarity:.2f}% similar)\n"

    return output

'''

Create a function that takes a book title as input, finds that book in the dataset, and then returns books with the highest cosine similarity scores to it.

In [5]:
def get_recommendations(title, df, cosine_sim=cosine_sim):
    # Ensure the title exists in the dataset to avoid errors
    if title not in df['Title'].values:
        return "The book title you entered is not in the dataset."
    
    # Get the index of the book that matches the title
    idx = df.index[df['Title'] == title].tolist()[0]

    # Get the pairwise similarity scores of all books with that book
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the books based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Skip the first one because it's the book itself and get the scores of the top 3 similar books
    top_sim_scores = sim_scores[1:4]

    # Get the book indices
    book_indices = [i[0] for i in top_sim_scores]
    similarities = [i[1] for i in top_sim_scores]

    input_book_categories = set(df.iloc[idx]['Categories'].split())

    # Format the output
    output = "Based on this book, the top 3 most closely related ones are:\n"
    for i, (index, similarity) in enumerate(zip(book_indices, similarities)):
        percentage_similarity = similarity * 100
        recommended_book_categories = set(df.iloc[index]['Categories'].split())
        common_categories = input_book_categories.intersection(recommended_book_categories)
        common_categories_list = list(common_categories)[:2]  # Get up to 2 common categories
        common_categories_str = ", ".join(common_categories_list)
        output += f"{i+1}: {df.iloc[index]['Title']} ({percentage_similarity:.2f}% similar) - Matching categories: {common_categories_str}\n"

    return output


In [8]:
print(get_recommendations('The psychology of effective studyinghow to succeed in your degree ', df, cosine_sim))

Based on this book, the top 3 most closely related ones are:
1: The psychology of effective studying :how to succeed in your degree  (74.09% similar) - Matching categories: skills, Study
2: P.O.W.E.R. learning :strategies for success in college and life  (51.01% similar) - Matching categories: skills, Study
3: Excelling in college :strategies for success & reducing stress  (31.90% similar) - Matching categories: skills, Study

